## Code Modules

In [1]:
import numpy as np,pandas as pd
import h5py,pylab as pl
import tensorflow_hub as th,tensorflow as tf
from tensorflow import image as timage

## Data

In [2]:
fpath='../input/tf-cats-vs-dogs/'
f='CatDogImages.h5'
f=h5py.File(fpath+f,'r')
keys=list(f.keys()); print(keys)
x_test=np.array(f[keys[0]])
y_test=np.array(f[keys[1]],dtype='int8')
x_train=np.array(f[keys[2]])
y_train=np.array(f[keys[3]],dtype='int8')
N=len(y_train); shuffle_ids=np.arange(N)
np.random.RandomState(12).shuffle(shuffle_ids)
x_train,y_train=x_train[shuffle_ids],y_train[shuffle_ids]
N=len(y_test); shuffle_ids=np.arange(N)
np.random.RandomState(23).shuffle(shuffle_ids)
x_test,y_test=x_test[shuffle_ids],y_test[shuffle_ids]
n=int(len(x_test)/2)
x_valid,y_valid=x_test[:n],y_test[:n]
x_test,y_test=x_test[n:],y_test[n:]
del f
pd.DataFrame([[x_train.shape,x_valid.shape,x_test.shape],
              [x_train.dtype,x_valid.dtype,x_test.dtype],
              [y_train.shape,y_valid.shape,y_test.shape],
              [y_train.dtype,y_valid.dtype,y_test.dtype]],
             columns=['train','valid','test'],
             index=['image shape','image type',
                    'label shape','label type'])

['test_images', 'test_labels', 'train_images', 'train_labels']


,train,valid,test
image shape,"(18610, 128, 128, 3)","(2326, 128, 128, 3)","(2326, 128, 128, 3)"
image type,float32,float32,float32
label shape,"(18610, 1)","(2326, 1)","(2326, 1)"
label type,int8,int8,int8


## TF Hub Models

In [3]:
fw='weights.best.hdf5'
def premodel(pix,den,mh,lbl,activ,loss):
    model=tf.keras.Sequential([
        tf.keras.layers.Input((pix,pix,3),
                              name='input'),
        th.KerasLayer(mh,trainable=True),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(den,activation='relu'),
        tf.keras.layers.Dropout(rate=.5),
        tf.keras.layers.Dense(lbl,activation=activ)])
    model.compile(optimizer='adam',
                  metrics=['accuracy'],loss=loss)
    display(model.summary())
    return model
def cb(fw):
    early_stopping=tf.keras.callbacks\
    .EarlyStopping(monitor='val_loss',patience=20,verbose=2)
    checkpointer=tf.keras.callbacks\
    .ModelCheckpoint(filepath=fw,save_best_only=True,verbose=2)
    lr_reduction=tf.keras.callbacks\
    .ReduceLROnPlateau(monitor='val_loss',verbose=2,
                       patience=5,factor=.8)
    return [checkpointer,early_stopping,lr_reduction]

In [4]:
[handle_base,pixels]=["inception_v3",128]
mhandle="https://tfhub.dev/google/imagenet/{}/classification/4"\
.format(handle_base)

In [5]:
model=premodel(pixels,1024,mhandle,1,
               'sigmoid','binary_crossentropy')
history=model.fit(x=x_train,y=y_train,batch_size=128,
                  epochs=5,callbacks=cb(fw),
                  validation_data=(x_valid,y_valid))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              23853833  
_________________________________________________________________
flatten (Flatten)            (None, 1001)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1026048   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 24,880,906
Trainable params: 24,846,474
Non-trainable params: 34,432
_________________________________________________________________


None

Epoch 1/5
146/146 [==============================] - ETA: 0s - loss: 0.6670 - accuracy: 0.8270
Epoch 00001: val_loss improved from inf to 8.17650, saving model to weights.best.hdf5
146/146 [==============================] - 33s 224ms/step - loss: 0.6670 - accuracy: 0.8270 - val_loss: 8.1765 - val_accuracy: 0.9058 - lr: 0.0010
Epoch 2/5
146/146 [==============================] - ETA: 0s - loss: 0.3909 - accuracy: 0.9566
Epoch 00002: val_loss improved from 8.17650 to 0.54738, saving model to weights.best.hdf5
146/146 [==============================] - 30s 206ms/step - loss: 0.3909 - accuracy: 0.9566 - val_loss: 0.5474 - val_accuracy: 0.9076 - lr: 0.0010
Epoch 3/5
146/146 [==============================] - ETA: 0s - loss: 0.9184 - accuracy: 0.8153
Epoch 00003: val_loss did not improve from 0.54738
146/146 [==============================] - 29s 198ms/step - loss: 0.9184 - accuracy: 0.8153 - val_loss: 321199.9062 - val_accuracy: 0.4979 - lr: 0.0010
Epoch 4/5
146/146 [=======================

In [6]:
model.load_weights(fw)
model.evaluate(x_test,y_test)

73/73 [==============================] - 2s 27ms/step - loss: 0.5350 - accuracy: 0.9127


[0.5349528193473816, 0.9127256870269775]